In [2]:
from gurobipy import Model , GRB , quicksum

In [26]:
m = Model("vaccine_distribution")

In [27]:
# Values
stations = ['A', 'B', 'C', 'D']
capacities = [2300, 1700, 1200, 900]
priorities = [0.5, 0.7, 1, 0.8]
total_vaccines = 5000

In [28]:
# 𝑋𝑖 : is the number of vaccines allocated to station 𝑖, ∀𝑖 ∈ {𝐴, 𝐵, 𝐶, 𝐷}
# x is a list of the 𝑋𝑖
x = [m.addVar(vtype=GRB.INTEGER, name=f"x_{station}") for station in stations]

# 𝑀𝑎𝑥𝑖𝑚𝑖𝑧𝑒 ∑ 𝑃𝑖 ⋅𝑋𝑖
m.setObjective(quicksum(x[i] * priorities[i] for i in range(len(stations))), GRB.MAXIMIZE)

In [29]:
# Every vaccine has to be used : ∑ 𝑋𝑖 = 5000
m.addConstr(quicksum(x[i] for i in range(len(stations))) == total_vaccines,"total_vaccines")

# Minimum supply : 𝑋𝑖 ≥ 500, ∀𝑖 ∈ {𝐴,𝐵,𝐶,𝐷}
# Storage space : 𝑋𝑖 ≤ 𝑆𝑖, ∀𝑖 ∈ {𝐴,𝐵,𝐶,𝐷}
for i in range(len(stations)):
    m.addConstr(x[i] >= 500, f"min_vaccines_{stations[i]}")
    m.addConstr(x[i] <= capacities[i], f"capacity_{stations[i]}")

In [30]:
m.setParam('OutputFlag', 0)
m.optimize()

if m.status == GRB.OPTIMAL:
    for i in range(len(stations)):
        print(f"Station {stations[i]} receives {x[i].x} vaccines") 
else :
    print("No optimal solution found")

Station A receives 1200.0 vaccines
Station B receives 1700.0 vaccines
Station C receives 1200.0 vaccines
Station D receives 900.0 vaccines
